In [ ]:
import numpy as np
from numpy.linalg import inv, solve

In [ ]:
np.random.seed(10)
M = 100 # number of training sets
N = 500 # number of samples in a training set
D = 6 # polynomial order
mu_x, mu_e, sigma_x, sigma_e = 1, 0, 1, 1 # distribution parameters
beta = np.array([1, 3, 6, 4, 5, 7]) # true coefficients
Y, X, E = np.zeros((N, M)), np.zeros((N, M)), np.zeros((N, M)) # response, predictor and error containers

In [ ]:
# Simulate data
for j in range(M):
    x = np.random.normal(mu_x, sigma_x, N)
    e = np.random.normal(mu_e, sigma_e, N)
    Y[:, j] = beta[0] * x**0 + beta[1] * x**1 + beta[2] * x**2 + beta[3] * x**3 + beta[4] * x**4 + beta[5] * x**5 + e
    X[:, j] = x
    E[:, j] = e

In [ ]:
# Fit regressions and calculate statistics
Err_in, Err_tr, omega, Cov, C_p = dict(), dict(), dict(), dict(), dict()
for d in range(D):
    err_tr, err_in = np.zeros(M), np.zeros(M)
    Y_hat_d = np.zeros((N, M))
    cov_y = np.zeros(N)
    for j in range(M):
        L_0_j = np.zeros((N, N))
        X_j = np.zeros((N, d + 1))
        for k in range(X_j.shape[1]):
            X_j[:, k] = X[:, j]**k
        b_j = np.dot(inv(np.dot(X_j.T, X_j)), np.dot(X_j.T, Y[:, [j]]))
        y_hat_j = np.dot(X_j, b_j).flatten()
        y_j = Y[:, j]
        for i in range(N):
            L_0_j[:, i] = (y_j - y_hat_j)**2
            y_j = np.concatenate((y_j[1:], y_j[:1]))
        err_in[j] = np.mean(np.mean(L_0_j, axis=0))
        err_tr[j] = np.mean(L_0_j[:, 0])
        Y_hat_d[:, j] = y_hat_j
    Err_in[d], Err_tr[d] = np.mean(err_in), np.mean(err_tr)
    omega[d] = Err_in[d] - Err_tr[d]
    C_p[d] = err_tr[0] + 2 * (d+1) * np.var(Y[:, 0] - Y_hat_d[:, 0]) / float(N)
    for i in range(N):
        cov_y[i] = np.cov(Y_hat_d[i, :], Y[i, :])[0, 1]
    Cov[d] = cov_y

In [ ]:
# Checking equation 7.21
print('Checking equation 7.21\n')
for d in range(D):
    lhs = omega[d]
    rhs = 2 * np.mean(Cov[d])
    print('%d parameter(s): lhs = %.2f, rhs = %.2f' %(d+1, lhs, rhs))


In [ ]:
# Checking equation 7.26 with effective number of parameters
print('\nChecking equation 7.23 using effective number of parameters\n')
for d in range(D):
    lhs = Err_in[d]
    rhs = C_p[d]
    print('%d parameter(s): lhs = %.2f, rhs = %.2f' %(d+1, lhs, rhs))